In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [3]:
# trainSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
# devSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
# testSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

trainSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.train.bio')
devSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testa.bio')
testSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testb.bio')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


12152
2867
3005


In [4]:
print(testSentences[3])

[['Die', 'O'], ['sechs', 'O'], ['Abgeordneten', 'O'], ['der', 'O'], ['Grünen', 'B-ORG'], [',', 'O'], ['die', 'O'], ['seit', 'O'], ['Bildung', 'O'], ['einer', 'O'], ['Großen', 'O'], ['Koalition', 'O'], ['von', 'O'], ['Christ-', 'O'], ['und', 'O'], ['Sozialdemokraten', 'O'], ['die', 'O'], ['Opposition', 'O'], ['darstellen', 'O'], [',', 'O'], ['hatten', 'O'], ['sich', 'O'], ['beim', 'O'], ['Vorsteher', 'O'], ['Rainer', 'B-PER'], ['Bergert', 'I-PER'], ['für', 'O'], ['ihr', 'O'], ['Fehlen', 'O'], ['entschuldigen', 'O'], ['lassen', 'O'], ['.', 'O']]


In [5]:
labelSet = set()
characters= set()
models.max_sequence_length = 0
n_gt_100 = 0
for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)
        if len(sentence) > 100:
            n_gt_100 += 1
print(n_gt_100)

0


In [6]:
print(len(labelSet))
print(models.max_sequence_length)

9
100


In [7]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [8]:
print(models.label2Idx)

{'B-ORG': 1, 'I-PER': 2, 'PADDING_TOKEN': 0, 'O': 6, 'B-PER': 4, 'I-ORG': 7, 'B-MISC': 3, 'B-LOC': 8, 'I-LOC': 5, 'I-MISC': 9}


In [9]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [10]:
print(models.case2Idx)

{'contains_digit': 7, 'PADDING_TOKEN': 0, 'allUpper': 3, 'other': 5, 'numeric': 1, 'initialUpper': 4, 'allLower': 2, 'mainly_numeric': 6}


In [11]:
print(type(trainSentences))

<class 'list'>


In [12]:
print(trainSentences[0])

[['Ereignis', 'O'], ['und', 'O'], ['Erzählung', 'O'], ['oder', 'O'], [':', 'O']]


In [13]:
models.char2Idx={"PADDING_TOKEN":0, "<S>":1, "</S>":2, "<W>":3, "</W>":4}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'<S>': 1, '7': 6, '<W>': 3, 'a': 62, 'J': 63, ')': 7, 'x': 5, '/': 9, 'v': 87, 'Q': 11, '=': 14, '3': 12, '-': 13, 'T': 60, 'P': 61, 'I': 69, 'A': 65, 'ì': 15, '4': 66, 'é': 16, 'ô': 17, 'p': 67, ',': 18, '!': 68, 'b': 95, '§': 70, 'O': 19, 'ñ': 72, 'ü': 73, '%': 20, 'q': 21, 'ç': 22, '9': 74, 'ä': 23, 'W': 24, 'c': 25, 'ö': 26, 'U': 27, 'V': 8, 'D': 76, '&': 77, 'u': 78, 'd': 28, 't': 29, 'í': 79, '"': 30, 's': 31, 'ß': 32, 'z': 80, '8': 83, 'Y': 34, 'S': 81, 'e': 35, '</S>': 2, 'N': 36, 'X': 37, '5': 38, '</W>': 4, 'l': 39, 'G': 41, 'PADDING_TOKEN': 0, 'f': 42, 'Z': 44, 'n': 85, 'E': 86, 'Ö': 88, '2': 89, 'ê': 90, '*': 82, '(': 45, 'M': 46, 'L': 47, 'j': 48, '+': 91, '6': 49, 'g': 40, '1': 92, 'Ü': 50, 'm': 57, '0': 102, 'i': 84, 'h': 93, 'B': 51, ';': 52, "'": 94, ':': 53, 'Ä': 54, 'K': 33, 'á': 55, 'r': 96, 'y': 56, 'è': 43, '?': 97, 'F': 98, 'H': 64, 'UNKNOWN': 103, 'ò': 71, 'R': 58, 'w': 99, 'o': 100, '.': 101, 'k': 75, 'C': 10, 'ó': 59}


In [14]:
print(trainSentences[0]) 

[['Ereignis', 'O'], ['und', 'O'], ['Erzählung', 'O'], ['oder', 'O'], [':', 'O']]


In [15]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [16]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
5


In [17]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

10
10


# Test Model

In [ ]:
# import importlib
# importlib.reload(models)

In [ ]:
tmp_model_filename = 'model_lstm_conll.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_lstm()

In [ ]:
#import importlib
#importlib.reload(utils)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences[:1000], shuffle_data=True, batch_size=256), 
    validation_data = utils.NerSequence(devSentences[:1000], batch_size=256), 
    epochs = 3, callbacks = [history]
)

In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 5, callbacks = [history]
)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences[:1000])
print(compute_f1(pred_labels, true_labels, models.idx2Label))

# Experiments

In [ ]:
f = open('results_lstm_conll.txt', 'a')
for run_i in range(10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'model_lstm_conll.' + str(run_i) + '.h5'

    history = utils.F1History(tmp_model_filename, devSet=devSentences)

    model = models.get_model_lstm()
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=32), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=256), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    true_labels, pred_labels = utils.predict_sequences(model, testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, models.idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

Run 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 3328        char_input[0][0]                 
__________________________________________________________________________________________________
words_input (InputLayer)        (None, None, 300)    0                                            
__________________________________________________________________________________________________
case

Epoch 1/10
379/380 [============================>.] - ETA: 0s - loss: 0.0394 - acc: 0.9900
New maximum F1 score: 0.7657776604172168 (before: 0) Saving to model_lstm_conll.1.h5
380/380 [==============================] - 250s 658ms/step - loss: 0.0393 - acc: 0.9900 - val_loss: 0.0212 - val_acc: 0.9942
Epoch 2/10
379/380 [============================>.] - ETA: 0s - loss: 0.0140 - acc: 0.9958
New maximum F1 score: 0.7961579509071505 (before: 0.7657776604172168) Saving to model_lstm_conll.1.h5
380/380 [==============================] - 241s 634ms/step - loss: 0.0139 - acc: 0.9959 - val_loss: 0.0159 - val_acc: 0.9948
Epoch 3/10
379/380 [============================>.] - ETA: 0s - loss: 0.0086 - acc: 0.9965
New maximum F1 score: 0.8174667751559533 (before: 0.7961579509071505) Saving to model_lstm_conll.1.h5
380/380 [==============================] - 242s 638ms/step - loss: 0.0085 - acc: 0.9965 - val_loss: 0.0113 - val_acc: 0.9953
Epoch 4/10
379/380 [============================>.] - ETA: 0s -

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_3cnn_bi-lstm.h5', '../models/final_model_germeval_outer.h5')
with open("../models/final_model_germeval.indexes", "w") as f:
    json.dump([models.idx2Label, models.label2Idx, models.char2Idx, models.case2Idx], f)

In [ ]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

finalmodel = load_model('../models/final_model_germeval_outer.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
f = open('germeval_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = "\t".join([str(i_tok+1), tok[0], correctlabel, correctlabel, guessedlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close